In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data
data = pd.read_csv('Data-Copy1.csv')
if "Unnamed: 0" in data.columns:
    data.drop(columns=["Unnamed: 0"], inplace=True)

# Define semester labels
semester_labels = [f"SEM-{i}" for i in range(2, 9)]

# Feature Engineering
data['Avg_SGPA'] = data.iloc[:, :-2].mean(axis=1)
data['SGPA_Improvement'] = data.iloc[:, 1:-2].subtract(data.iloc[:, :-3].values).mean(axis=1)
data['SGPA_Difference'] = data.iloc[:, -3] - data.iloc[:, 0]

# Split data dynamically
x_train_test = {}
y_train_test = {}
scalers = {}
mlp_models = {}

for i in range(2, 9):
    predictors = semester_labels[: i - 1] + ["Avg_SGPA", "SGPA_Improvement", "SGPA_Difference"]
    target = f"SEM-{i}"
    
    x = data[predictors]
    y = data[target]

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2)
    x_train_test[target] = (x_train, x_test)
    y_train_test[target] = (y_train, y_test)

    # Standardization
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    scalers[target] = scaler

    # Train model
    model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=1000, random_state=2)
    model.fit(x_train_scaled, y_train)
    mlp_models[target] = model

# Evaluation function
def evaluate_model(model, x_test_scaled, y_test, sem_label):
    y_pred = model.predict(x_test_scaled)
    print(f"Model Performance for {sem_label}:")
    print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")
    print(f"MSE: {mean_squared_error(y_test, y_pred):.4f}")
    print(f"R² Score: {model.score(x_test_scaled, y_test):.4f}\n")

# Predict SGPA for a selected semester
ch = int(input("Enter the semester number you want to predict (2-8): "))

if ch in range(2, 9):
    target = f"SEM-{ch}"
    predictors = semester_labels[: ch - 1] + ["Avg_SGPA", "SGPA_Improvement", "SGPA_Difference"]
    evaluate_model(mlp_models[target], scalers[target].transform(x_train_test[target][1]), y_train_test[target][1], target)
    print("Enter your SGPA for previous semesters one by one:")
    prev_sgpa = [float(input(f"Enter SGPA for SEM-{i}: ")) for i in range(1, ch)]
    
    # Compute derived features
    avg_sgpa = np.mean(prev_sgpa)
    sgpa_improvement = np.mean(np.diff(prev_sgpa)) if len(prev_sgpa) > 1 else 0
    sgpa_difference = prev_sgpa[-1] - prev_sgpa[0] if len(prev_sgpa) > 1 else 0
    
    new_input_extended = np.array(prev_sgpa + [avg_sgpa, sgpa_improvement, sgpa_difference]).reshape(1, -1)

    # Standardize and Predict
    new_input_scaled = scalers[target].transform(new_input_extended)
    prediction = mlp_models[target].predict(new_input_scaled)

    print(f"Predicted SGPA for {target}: {round(prediction[0], 2)}")
else:
    print("Invalid choice. Please enter a number between 2 and 8.")


Enter the semester number you want to predict (2-8):  2


Model Performance for SEM-2:
MAE: 0.0530
MSE: 0.0054
R² Score: 0.9944

Enter your SGPA for previous semesters one by one:


Enter SGPA for SEM-1:  8.5


Predicted SGPA for SEM-2: 8.47
